In [16]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

# please use the same identificator that was used in signed certificate
client_id = 'frontend'
director_node_fqdn = 'localhost'
director_port = 50050

# 1) Run with API layer - Director mTLS 
# If the user wants to enable mTLS their must provide CA root chain, and signed key pair to the federation interface
# cert_chain = 'cert/root_ca.crt'
# API_certificate = 'cert/frontend.crt'
# API_private_key = 'cert/frontend.key'

# federation = Federation(
#     client_id=client_id,
#     director_node_fqdn=director_node_fqdn,
#     director_port=director_port,
#     tls=True,
#     cert_chain=cert_chain,
#     api_cert=api_certificate,
#     api_private_key=api_private_key
# )

# --------------------------------------------------------------------------------------------------------------------

# 2) Run with TLS disabled (trusted environment)
# Federation can also determine local fqdn automatically
federation = Federation(
    client_id=client_id,
    director_node_fqdn=director_node_fqdn,
    director_port=director_port,
    tls=False
)

In [17]:
shard_registry = federation.get_shard_registry()

In [18]:
federation.target_shape
dummy_shard_desc = federation.get_dummy_shard_descriptor(size=10)
dummy_shard_dataset = dummy_shard_desc.get_dataset('train')
sample, target = dummy_shard_dataset[0]
f"Sample shape: {sample.shape}, target shape: {target.shape}"

'Sample shape: (300, 400, 3), target shape: (300, 400)'

# Creating a FL experiment using Interactive API

In [19]:
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment


## Register dataset
We extract User dataset class implementation. Is it convinient? What if the dataset is not a class?

In [20]:
import os
import PIL
import numpy as np
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms as tsf


class KvasirShardDataset(Dataset):
    
    def __init__(self, dataset):
        self._dataset = dataset
        
        # Prepare transforms
        self.img_trans = tsf.Compose([
            tsf.ToPILImage(),
            tsf.Resize((332, 332)),
            tsf.ToTensor(),
            tsf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
        self.mask_trans = tsf.Compose([
            tsf.ToPILImage(),
            tsf.Resize((332, 332), interpolation=PIL.Image.NEAREST),
            tsf.ToTensor()])
        
    def __getitem__(self, index):
        img, mask = self._dataset[index]
        img = self.img_trans(img).numpy()
        mask = self.mask_trans(mask).numpy()
        return img, mask
    
    def __len__(self):
        return len(self._dataset)

    
# Now you can implement you data loaders using dummy_shard_desc
class KvasirSD(DataInterface):

    def __init__(self, validation_fraction=1/8, **kwargs):
        super().__init__(**kwargs)
        
        self.validation_fraction = validation_fraction
        
    @property
    def shard_descriptor(self):
        return self._shard_descriptor
        
    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor  will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor
        self._shard_dataset = KvasirShardDataset(shard_descriptor.get_dataset('train'))
        
        validation_size = max(1, int(len(self._shard_dataset) * self.validation_fraction))
        
        self.train_indeces = np.arange(len(self._shard_dataset) - validation_size)
        self.val_indeces = np.arange(len(self._shard_dataset) - validation_size, len(self._shard_dataset))
    
    def get_train_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        train_sampler = SubsetRandomSampler(self.train_indeces)
        return DataLoader(
            self._shard_dataset,
            num_workers=8,
            batch_size=self.kwargs['train_bs'],
            sampler=train_sampler
        )

    def get_valid_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks without optimizer in contract
        """
        val_sampler = SubsetRandomSampler(self.val_indeces)
        return DataLoader(
            self._shard_dataset,
            num_workers=8,
            batch_size=self.kwargs['valid_bs'],
            sampler=val_sampler
        )

    def get_train_data_size(self):
        """
        Information for aggregation
        """
        return len(self.train_indeces)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.val_indeces)

In [21]:
fed_dataset = KvasirSD(train_bs=4, valid_bs=8)
fed_dataset.shard_descriptor = dummy_shard_desc
for i, (sample, target) in enumerate(fed_dataset.get_train_loader()):
    print(sample.shape)

/home/cloud/anaconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/cloud/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([4, 3, 332, 332])
torch.Size([4, 3, 332, 332])
torch.Size([1, 3, 332, 332])


## Describe a model and optimizer


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

"""
UNet model definition
"""
from layers import soft_dice_coef, soft_dice_loss, DoubleConv, Down, Up


class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super().__init__()
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.up1 = Up(512, 256)
        self.up2 = Up(256, 128)
        self.up3 = Up(128, 64)
        self.outc = nn.Conv2d(64, n_classes, 1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x = self.up1(x4, x3)
        x = self.up2(x, x2)
        x = self.up3(x, x1)
        x = self.outc(x)
        x = torch.sigmoid(x)
        return x
    
model_unet = UNet()

from torchvision.models.segmentation import fcn_resnet50, deeplabv3_resnet50, deeplabv3_mobilenet_v3_large


In [23]:
model_resnet_deeplabv3 = deeplabv3_resnet50(pretrained=True, progress=False)

optimizer_adam = optim.Adam(model_unet.parameters(), lr=1e-4)

## Register model

In [24]:
from copy import deepcopy

framework_adapter = 'openfl.plugins.frameworks_adapters.pytorch_adapter.FrameworkAdapterPlugin'
MI = ModelInterface(model=model_unet, optimizer=optimizer_adam, framework_plugin=framework_adapter)
 
# Save the initial model state
initial_model = deepcopy(model_unet)

## Define and register FL tasks

In [25]:
TI = TaskInterface()
import torch

import tqdm
from openfl.component.aggregation_functions import Median

# The Interactive API supports registering functions definied in main module or imported.
def function_defined_in_notebook(some_parameter):
    print(f'Also I accept a parameter and it is {some_parameter}')

#The Interactive API supports overriding of the aggregation function
aggregation_function = Median()

# Task interface currently supports only standalone functions.
@TI.add_kwargs(**{'some_parameter': 42})
@TI.register_fl_task(model='unet_model', data_loader='train_loader', \
                     device='device', optimizer='optimizer')     
@TI.set_aggregation_function(aggregation_function)
def train(unet_model, train_loader, optimizer, device, loss_fn=soft_dice_loss, some_parameter=None):
    
    """    
    The following constructions, that may lead to resource race
    is no longer needed:
    
    if not torch.cuda.is_available():
        device = 'cpu'
    else:
        device = 'cuda'
        
    """

    print(f'\n\n TASK TRAIN GOT DEVICE {device}\n\n')
    
    function_defined_in_notebook(some_parameter)
    
    train_loader = tqdm.tqdm(train_loader, desc="train")
    
    unet_model.train()
    unet_model.to(device)

    losses = []

    for data, target in train_loader:
        data, target = torch.tensor(data).to(device), torch.tensor(
            target).to(device, dtype=torch.float32)
        optimizer.zero_grad()
        output = unet_model(data)
        loss = loss_fn(output=output, target=target)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
        
    return {'train_loss': np.mean(losses),}


@TI.register_fl_task(model='unet_model', data_loader='val_loader', device='device')     
def validate(unet_model, val_loader, device):
    print(f'\n\n TASK VALIDATE GOT DEVICE {device}\n\n')
    
    unet_model.eval()
    unet_model.to(device)
    
    val_loader = tqdm.tqdm(val_loader, desc="validate")

    val_score = 0
    total_samples = 0

    with torch.no_grad():
        for data, target in val_loader:
            samples = target.shape[0]
            total_samples += samples
            data, target = torch.tensor(data).to(device), \
                torch.tensor(target).to(device, dtype=torch.int64)
            output = unet_model(data)
            val = soft_dice_coef(output, target)
            val_score += val.sum().cpu().numpy()
            
    return {'dice_coef': val_score / total_samples,}

## Start federated learning experiment

In [26]:
# create an experimnet in federation
experiment_name = 'kvasir_test_experiment'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

In [27]:
# If I use autoreload I got a pickling error

# The following command zips the workspace and python requirements to be transfered to collaborator nodes
fl_experiment.start(model_provider=MI, 
                    task_keeper=TI,
                    data_loader=fed_dataset,
                    rounds_to_train=2,
                    opt_treatment='CONTINUE_GLOBAL',
                    device_assignment_policy='CUDA_PREFERRED')

In [28]:
# If user want to stop IPython session, then reconnect and check how experiment is going 
# fl_experiment.restore_experiment_state(MI)
fl_experiment.stream_metrics()


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1652344061.937244442","description":"Error received from peer ipv4:127.0.0.1:50050","file":"src/core/lib/surface/call.cc","file_line":1062,"grpc_message":"Socket closed","grpc_status":14}"
>

In [41]:
best_model = fl_experiment.get_best_model()

In [23]:
fl_experiment.remove_experiment_data()


In [42]:
best_model.inc.conv[0].weight


Parameter containing:
tensor([[[[ 0.0700,  0.1357,  0.0769],
          [-0.1708,  0.1554, -0.0682],
          [-0.0234, -0.1518,  0.1711]],

         [[-0.0013, -0.1897,  0.0597],
          [ 0.0030, -0.0331, -0.1827],
          [ 0.0577,  0.1629, -0.0462]],

         [[-0.1714, -0.1575, -0.1074],
          [ 0.1422,  0.1916, -0.0799],
          [ 0.0362, -0.0464, -0.1664]]],


        [[[-0.0115, -0.0973,  0.1385],
          [-0.0702, -0.0377,  0.1049],
          [-0.0385,  0.1891,  0.0289]],

         [[ 0.0349,  0.0471,  0.1921],
          [-0.0776,  0.1406,  0.1020],
          [ 0.0596, -0.0203, -0.1517]],

         [[ 0.0203,  0.1058,  0.0203],
          [-0.1727,  0.0744,  0.0993],
          [ 0.0628, -0.0073, -0.0023]]],


        [[[ 0.0288,  0.0536, -0.1757],
          [-0.1652,  0.1441, -0.1404],
          [ 0.0129,  0.1680,  0.1551]],

         [[-0.0479,  0.0066, -0.1390],
          [ 0.0912,  0.0539,  0.1425],
          [ 0.0506,  0.1185,  0.1592]],

         [[-0.0094,  0

In [43]:
# validating initial model
validate(initial_model, fed_dataset.get_valid_loader(), 'cpu')


validate:   0%|          | 0/1 [00:00<?, ?it/s]



 TASK VALIDATE GOT DEVICE cpu




<ipython-input-37-8df218eba781>:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = torch.tensor(data).to(device), \
<ipython-input-37-8df218eba781>:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(target).to(device, dtype=torch.int64)
validate: 100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


{'dice_coef': 3.4458509617252275e-05}

In [44]:
# Validating trained model
validate(best_model, fed_dataset.get_valid_loader(), 'cpu')

validate:   0%|          | 0/1 [00:00<?, ?it/s]



 TASK VALIDATE GOT DEVICE cpu




<ipython-input-37-8df218eba781>:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = torch.tensor(data).to(device), \
<ipython-input-37-8df218eba781>:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(target).to(device, dtype=torch.int64)
validate: 100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


{'dice_coef': 3.4458509617252275e-05}

# We can tune model further!

In [45]:
MI = ModelInterface(model=best_model, optimizer=optimizer_adam, framework_plugin=framework_adapter)
fl_experiment.start(model_provider=MI, task_keeper=TI, data_loader=fed_dataset, rounds_to_train=4, \
                              opt_treatment='CONTINUE_GLOBAL')

In [46]:
best_model = fl_experiment.get_best_model()
# Validating trained model
validate(best_model, fed_dataset.get_valid_loader(), 'cpu')

validate:   0%|          | 0/1 [00:00<?, ?it/s]



 TASK VALIDATE GOT DEVICE cpu




<ipython-input-37-8df218eba781>:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = torch.tensor(data).to(device), \
<ipython-input-37-8df218eba781>:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(target).to(device, dtype=torch.int64)
validate: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


{'dice_coef': 3.4458509617252275e-05}